In [6]:
import json
import tfx
import base64
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [1]:
DATA_ROOT = 'data/combined_data.csv'

In [8]:
emails = pd.read_csv(DATA_ROOT)
emails.head(10)

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
5,0,larry king live at escapenumber escapenumber p...
6,0,michael pobega wrote i'm not sure if it's the ...
7,0,hi i have this error tr sample escapenumber es...
8,1,works gateway world art explore tattooing full...
9,1,upon this account he is not only very cautious...


In [10]:
X, y = emails['text'], emails['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=12)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(66758,) (16690,) (66758,) (16690,)


In [49]:
test_email = X_val[58992]
test_email_prediction = y_val[58992]

label_names = ['Legitimate', 'Spam']

print("Text:\n", test_email)
print("Label: ", label_names[test_email_prediction])

Text:
 bull market report target sym bvyhprice current escapenumber escapenumber escapenumber day target price escapenumber escapenumberaction strong buy hold insider buying alert short term kst check the news of bvyh manager contact broker 
Label:  Spam


In [52]:
example = tf.train.Example(features=tf.train.Features(feature={
    'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[test_email.encode('utf-8')]))
}))

serialized_example = example.SerializeToString()

json_data = json.dumps(
    {
        "signature_name":"serving_default",
        "inputs": [{"b64": base64.b64encode(serialized_example).decode('utf-8')}],
    }
)

endpoint = "https://spam-email-project-production.up.railway.app/v1/models/se-model:predict"

response = requests.post(endpoint, data=json_data)
prediction = response.json()['outputs'][0][0]

if prediction > 0.5:
    print("Spam")
else:
    print("Legitimate")

Spam
